In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions-dataset-for-nlp/val.txt
/kaggle/input/emotions-dataset-for-nlp/test.txt
/kaggle/input/emotions-dataset-for-nlp/train.txt


In [3]:
!pip install text_hammer
!pip install transformers
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False # if autocompletion doesnot work in kaggle notebook | hit tab
import text_hammer as th
import numpy as np
import pandas as pd
import seaborn as sns
from array import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from transformers import BertTokenizer,TFBertModel
#tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

tokenizer.save_pretrained('bert-tokenizer')
bert.save_pretrained('bert-model')
import shutil
shutil.make_archive('bert-tokenizer', 'zip', 'bert-tokenizer')
shutil.make_archive('bert-model','zip','bert-model')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

def batch_selector(dataset,samples_no):
    training_batch=dataset.iloc[:samples_no]
    unlabelled_batch=dataset.iloc[samples_no:]
    return training_batch, unlabelled_batch
def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
#     df[column] = df[column].progress_apply(lambda x: ps.remove_stopwords(x))

    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
#     df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    return(df)

def num_sim(n1, n2):
  """ calculates a similarity score between 2 numbers """
  return 1 - abs(n1 - n2) / (n1 + n2)

def maxConflictFinder(array):
    a_list = list(array)
    are= list(array)
    max_value = max(a_list)
    max_index = a_list.index(max_value)
    maxele=a_list.pop(max_index)
    for i in range(len(a_list)):
        intdiff=num_sim(a_list[i],maxele)
        if((intdiff>0.95)):
            return are.index(a_list[i]),are.index(maxele)
        else:
            return None

def indexGenerator(df):
    n,_=df.shape
    index=np.arange(0,n)
    df['index']=index
    df=df.set_index('index')
    return df

def dataTx(n,df_test1,df_train1):
    df_train1=df_trai1n.append(df_test1[n:n+1])
    df_test1=df_test1.drop(n)
    return df_test1,df_train1

import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.1
    Uninstalling beautifulsoup4-4.11.1:
      Successfully uninstalled beautifulsoup4-4.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompati

2022-11-17 15:41:53.003578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 15:41:53.004629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 15:41:53.005842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 15:41:53.006695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 15:41:53.007495: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

2022-11-17 15:43:09.092904: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the 

In [4]:
df_train = pd.read_csv('../input/emotions-dataset-for-nlp/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test = pd.read_csv('../input/emotions-dataset-for-nlp/test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')
df_val=pd.read_csv('../input/emotions-dataset-for-nlp/val.txt',header=None,sep=';',names=['Input','Sentiment'],encoding='utf-8')
tweet_df = pd.concat([df_train,df_test,df_val], axis = 0)
df_surprise=tweet_df[tweet_df['Sentiment']=='surprise']
df_fear    =tweet_df[tweet_df['Sentiment']=='fear']
df_joy     =tweet_df[tweet_df['Sentiment']=='joy']
df_love    =tweet_df[tweet_df['Sentiment']=='love']
df_anger   =tweet_df[tweet_df['Sentiment']=='anger']
df_sadness =tweet_df[tweet_df['Sentiment']=='sadness']

print(f'sadness:{df_sadness.shape}')
print(f'anger:{df_anger.shape}')
print(f'love:{df_love.shape}')
print(f'joy:{df_joy.shape}')
print(f'fear:{df_fear.shape}')
print(f'surprise:{df_surprise.shape}')

No_samples=100
surprise_train,surprise_test=batch_selector(df_surprise,No_samples)
fear_train,fear_test=batch_selector(df_fear,No_samples)
joy_train,joy_test=batch_selector(df_joy,No_samples)
love_train,love_test=batch_selector(df_love,No_samples)
anger_train,anger_test=batch_selector(df_anger,No_samples)
sadness_train,sadness_test=batch_selector(df_sadness,No_samples)

test250 = pd.concat([surprise_test,fear_test,joy_test,love_test,anger_test,sadness_test], axis = 0)
train250= pd.concat([surprise_train,fear_train,joy_train,love_train,anger_train,sadness_train], axis = 0)


sadness:(5797, 2)
anger:(2709, 2)
love:(1641, 2)
joy:(6761, 2)
fear:(2373, 2)
surprise:(719, 2)


In [5]:
df_test,df_train=test250,train250
print(df_test.shape,df_train.shape)

(19400, 2) (600, 2)


In [6]:
# for k in range(100):
#     print('####################',k+1,"###########################")
#     df_cleaned_train_20 = text_preprocessing(df_train,'Input')
#     df_cleaned_test_20 = text_preprocessing(df_test,'Input')
#     train20_df = df_cleaned_train_20.copy()
#     test20_df = df_cleaned_test_20.copy()
#     train20_df['num_words'] = train20_df.Input.apply(lambda x:len(x.split()))
#     test20_df['num_words'] = test20_df.Input.apply(lambda x:len(x.split()))
#     train20_df['Sentiment'] = train20_df.Sentiment.astype('category')
#     test20_df['Sentiment'] = test20_df.Sentiment.astype('category')
#     train20_df['Sentiment']  =  train20_df.Sentiment.cat.codes
#     test20_df['Sentiment']  =  test20_df.Sentiment.cat.codes
#     from sklearn.model_selection import train_test_split
#     data_train,data_test = train_test_split(train20_df, test_size = 0.30, random_state = 42, stratify = train20_df.Sentiment)
#     surprise_string ='i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies '
#     len(surprise_string)
#     res = len(surprise_string.split())
#     print("The number of words in string are : " + str(res),len(surprise_string) )
#     data_test = data_test.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
#     data_train = data_train.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
#     data_val = test20_df.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
    
#     x_train = tokenizer(
#         text=data_train.Input.tolist(),
#         add_special_tokens=True,
#         max_length=70,
#         truncation=True,
#         padding=True, 
#         return_tensors='tf',
#         return_token_type_ids = False,
#         return_attention_mask = True,
#         verbose = True)
    
#     x_test = tokenizer(
#         text=data_test.Input.tolist(),
#         add_special_tokens=True,
#         max_length=70,
#         truncation=True,
#         padding=True, 
#         return_tensors='tf',
#         return_token_type_ids = False,
#         return_attention_mask = True,
#         verbose = True)
    
#     x_val = tokenizer(
#         text=data_val.Input.tolist(),
#         add_special_tokens=True,
#         max_length=70,
#         truncation=True,
#         padding=True, 
#         return_tensors='tf',
#         return_token_type_ids = False,
#         return_attention_mask = True,
#         verbose = True)
    
#     max_len = 70
#     import tensorflow as tf
#     from tensorflow.keras.layers import Input, Dense

#     input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
#     input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

#     embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
#     out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
#     out = Dense(128, activation='relu')(out)
#     out = tf.keras.layers.Dropout(0.1)(out)
#     out = Dense(32,activation = 'relu')(out)
#     y = Dense(6,activation = 'sigmoid')(out)
#     model4 = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
#     model4.layers[2].trainable = True
    
#     optimizer = Adam(
#         learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
#         epsilon=1e-08,
#         decay=0.01,
#         clipnorm=1.0)
#     loss =CategoricalCrossentropy(from_logits = True)
#     metric = CategoricalAccuracy('balanced_accuracy'),
#     model4.compile(
#         optimizer = optimizer,
#         loss = loss, 
#         metrics = metric)
#     tf.config.experimental_run_functions_eagerly(True)
#     tf.config.run_functions_eagerly(True)
#     train_history = model4.fit(
#         x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
#         y = to_categorical(data_train.Sentiment),
#         validation_data = (
#             {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.Sentiment)
#             ),
#        epochs=10,
#        batch_size=36
#        )
#     predicted_raw = model4.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})
#     ine,_=predicted_raw.shape
#     lis=[]
#     lis3=[]
#     df_test=indexGenerator(df_test)
#     df_train=indexGenerator(df_train)
#     for i in range(ine):
#         if (maxConflictFinder(predicted_raw[i]) != None):
#             lis.append(maxConflictFinder(predicted_raw[i]))
#             lis3.append(i)
            
#     df_test=indexGenerator(df_test)
#     df_train=indexGenerator(df_train)
    
#     for i in lis3:
#         df_test,df_train=dataTx(i,df_test,df_train)
    
#     print(df_test.shape,df_train.shape)
#     df_test=indexGenerator(df_test)
#     df_train=indexGenerator(df_train)

#     df_cleaned_train_20 = text_preprocessing(df_train,'Input')
#     df_cleaned_test_20 = text_preprocessing(df_test,'Input')
#     train20_df = df_cleaned_train_20.copy()
#     test20_df = df_cleaned_test_20.copy()
#     train20_df['num_words'] = train20_df.Input.apply(lambda x:len(x.split()))
#     test20_df['num_words'] = test20_df.Input.apply(lambda x:len(x.split()))
#     train20_df['Sentiment'] = train20_df.Sentiment.astype('category')
#     test20_df['Sentiment'] = test20_df.Sentiment.astype('category')
#     train20_df['Sentiment']  =  train20_df.Sentiment.cat.codes
#     test20_df['Sentiment']  =  test20_df.Sentiment.cat.codes

#     from sklearn.model_selection import train_test_split
#     data_train,data_test = train_test_split(train20_df, test_size = 0.30, random_state = 42, stratify = train20_df.Sentiment)

#     surprise_string ='i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies '
#     len(surprise_string)
#     res = len(surprise_string.split())
#     print("The number of words in string are : " + str(res),len(surprise_string) )
#     data_test = data_test.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
#     data_train = data_train.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
#     data_val = test20_df.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
    
#     x_train = tokenizer(
#         text=data_train.Input.tolist(),
#         add_special_tokens=True,
#         max_length=70,
#         truncation=True,
#         padding=True, 
#         return_tensors='tf',
#         return_token_type_ids = False,
#         return_attention_mask = True,
#         verbose = True)

#     x_test = tokenizer(
#         text=data_test.Input.tolist(),
#         add_special_tokens=True,
#         max_length=70,
#         truncation=True,
#         padding=True, 
#         return_tensors='tf',
#         return_token_type_ids = False,
#         return_attention_mask = True,
#         verbose = True)

#     x_val = tokenizer(
#         text=data_val.Input.tolist(),
#         add_special_tokens=True,
#         max_length=70,
#         truncation=True,
#         padding=True, 
#         return_tensors='tf',
#         return_token_type_ids = False,
#         return_attention_mask = True,
#         verbose = True)
    
#     max_len = 70
#     import tensorflow as tf
#     from tensorflow.keras.layers import Input, Dense

#     input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
#     input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

#     embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
#     out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
#     out = Dense(128, activation='relu')(out)
#     out = tf.keras.layers.Dropout(0.1)(out)
#     out = Dense(32,activation = 'relu')(out)

#     y = Dense(6,activation = 'sigmoid')(out)
    
#     model5 = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
#     model5.layers[2].trainable = True
#     optimizer = Adam(
#         learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
#         epsilon=1e-08,
#         decay=0.01,
#         clipnorm=1.0)
#     loss =CategoricalCrossentropy(from_logits = True)
#     metric = CategoricalAccuracy('balanced_accuracy'),
#     model5.compile(
#         optimizer = optimizer,
#         loss = loss, 
#         metrics = metric)
#     tf.config.experimental_run_functions_eagerly(True)
#     tf.config.run_functions_eagerly(True)

#     train_history = model5.fit(
#         x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
#         y = to_categorical(data_train.Sentiment),
#         validation_data = (
#             {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.Sentiment)
#             ),
#        epochs=10,
#        batch_size=36
#        )
#     predicted_raw = model5.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})
#     predicted_raw = model5.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})
#     y_predicted = np.argmax(predicted_raw, axis = 1)
#     print(classification_report(data_val.Sentiment, y_predicted))
#     print(confusion_matrix(data_val.Sentiment,y_predicted))
#     cf_matrix=confusion_matrix(data_val.Sentiment,y_predicted)
#     if(len(lis3)==0):
#         break


In [7]:
def dataTx_trial(txlist,df_test1,df_train1):
    for n in txlist:
        df_train1=df_train1.append(indexGenerator((df_test1.iloc[n].to_frame()).transpose()))
    for k in txlist:
        df_test1=df_test1.drop(k)
    return df_test1,df_train1

df_test,df_train=test250,train250
print(df_test.shape,df_train.shape)

(19400, 2) (600, 2)


In [ ]:
ine,_=predicted_raw.shape
lis=[]
lis3=[]
df_test=indexGenerator(df_test)
df_train=indexGenerator(df_train)

for i in range(ine):
    if (maxConflictFinder(predicted_raw[i]) != None):
        lis3.append(i)
    else:
        lis.append(i)
        
print(len(lis3))
df_test=indexGenerator(df_test)
df_train=indexGenerator(df_train)
        
df_test,df_train=dataTx_trial(lis3,df_test,df_train)
    
df_test=indexGenerator(df_test)
df_train=indexGenerator(df_train)

In [8]:
print(df_test.shape,df_train.shape)


(19400, 2) (600, 2)


In [9]:
df_cleaned_train_20 = text_preprocessing(df_train,'Input')
df_cleaned_test_20 = text_preprocessing(df_test,'Input')
train20_df = df_cleaned_train_20.copy()
test20_df = df_cleaned_test_20.copy()
train20_df['num_words'] = train20_df.Input.apply(lambda x:len(x.split()))
test20_df['num_words'] = test20_df.Input.apply(lambda x:len(x.split()))
train20_df['Sentiment'] = train20_df.Sentiment.astype('category')
test20_df['Sentiment'] = test20_df.Sentiment.astype('category')
train20_df['Sentiment']  =  train20_df.Sentiment.cat.codes
test20_df['Sentiment']  =  test20_df.Sentiment.cat.codes

from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(train20_df, test_size = 0.30, random_state = 42, stratify = train20_df.Sentiment)

surprise_string ='i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies '
len(surprise_string)
res = len(surprise_string.split())
print("The number of words in string are : " + str(res),len(surprise_string) )
data_test = data_test.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
data_train = data_train.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
data_val = test20_df.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)

x_train = tokenizer(
    text=data_train.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

x_test = tokenizer(
    text=data_test.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

x_val = tokenizer(
    text=data_val.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

max_len = 70
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(6,activation = 'sigmoid')(out)
    
model5 = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model5.layers[2].trainable = True

optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model5.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

train_history = model5.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train.Sentiment),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.Sentiment)
    ),
  epochs=10,
    batch_size=36
)

predicted_raw = model5.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})
y_predicted = np.argmax(predicted_raw, axis = 1)
print(classification_report(data_val.Sentiment, y_predicted))
print(confusion_matrix(data_val.Sentiment,y_predicted))
cf_matrix=confusion_matrix(data_val.Sentiment,y_predicted)

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

...

  0%|          | 0/19400 [00:00<?, ?it/s]

  0%|          | 0/19400 [00:00<?, ?it/s]

The number of words in string are : 115 565


2022-11-17 15:45:00.984737: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
12/12 [==============================] - 9s 725ms/step - loss: 1.8898 - balanced_accuracy: 0.1924 - val_loss: 1.7950 - val_balanced_accuracy: 0.2099
Epoch 2/10
12/12 [==============================] - 9s 760ms/step - loss: 1.6900 - balanced_accuracy: 0.2945 - val_loss: 1.6899 - val_balanced_accuracy: 0.3039
Epoch 3/10
12/12 [==============================] - 9s 755ms/step - loss: 1.4873 - balanced_accuracy: 0.4632 - val_loss: 1.5227 - val_balanced_accuracy: 0.4365
Epoch 4/10
12/12 [==============================] - 10s 807ms/step - loss: 1.1878 - balanced_accuracy: 0.6105 - val_loss: 1.3096 - val_balanced_accuracy: 0.5249
Epoch 5/10
12/12 [==============================] - 9s 789ms/step - loss: 0.8554 - balanced_accuracy: 0.7767 - val_loss: 1.1151 - val_balanced_accuracy: 0.5856
Epoch 6/10
12/12 [==============================] - 9s 760ms/step - loss: 0.5265 - balanced_accuracy: 0.9002 - val_loss: 0.9322 - val_balanced_accuracy: 0.6685
Epoch 7/10
12/12 [=====================

In [ ]:
df_test,df_train=test250,train250
print(df_test.shape,df_train.shape)

In [10]:
for k in range(100):
    print('####################',k+1,"###########################")
    ine,_=predicted_raw.shape
    lis=[]
    lis3=[]
    df_test=indexGenerator(df_test)
    df_train=indexGenerator(df_train)
    for i in range(ine):
        if (maxConflictFinder(predicted_raw[i]) != None):
            lis3.append(i)
    print(len(lis3))        
    df_test=indexGenerator(df_test)
    df_train=indexGenerator(df_train)
    df_test,df_train=dataTx_trial(lis3,df_test,df_train)
    print(df_test.shape,df_train.shape)
    df_test=indexGenerator(df_test)
    df_train=indexGenerator(df_train)

    df_cleaned_train_20 = text_preprocessing(df_train,'Input')
    df_cleaned_test_20 = text_preprocessing(df_test,'Input')
    train20_df = df_cleaned_train_20.copy()
    test20_df = df_cleaned_test_20.copy()
    train20_df['num_words'] = train20_df.Input.apply(lambda x:len(x.split()))
    test20_df['num_words'] = test20_df.Input.apply(lambda x:len(x.split()))
    train20_df['Sentiment'] = train20_df.Sentiment.astype('category')
    test20_df['Sentiment'] = test20_df.Sentiment.astype('category')
    train20_df['Sentiment']  =  train20_df.Sentiment.cat.codes
    test20_df['Sentiment']  =  test20_df.Sentiment.cat.codes

    from sklearn.model_selection import train_test_split
    data_train,data_test = train_test_split(train20_df, test_size = 0.30, random_state = 42, stratify = train20_df.Sentiment)

    surprise_string ='i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies '
    len(surprise_string)
    res = len(surprise_string.split())
    print("The number of words in string are : " + str(res),len(surprise_string) )
    data_test = data_test.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
    data_train = data_train.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
    data_val = test20_df.append({'Input':surprise_string, 'Sentiment':1, 'num_words':115}, ignore_index=True)
    
    x_train = tokenizer(
        text=data_train.Input.tolist(),
        add_special_tokens=True,
        max_length=70,
        truncation=True,
        padding=True, 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True)

    x_test = tokenizer(
        text=data_test.Input.tolist(),
        add_special_tokens=True,
        max_length=70,
        truncation=True,
        padding=True, 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True)

    x_val = tokenizer(
        text=data_val.Input.tolist(),
        add_special_tokens=True,
        max_length=70,
        truncation=True,
        padding=True, 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True)
    
    max_len = 70
    import tensorflow as tf
    from tensorflow.keras.layers import Input, Dense

    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
    out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    out = Dense(128, activation='relu')(out)
    out = tf.keras.layers.Dropout(0.1)(out)
    out = Dense(32,activation = 'relu')(out)

    y = Dense(6,activation = 'sigmoid')(out)
    
    model5 = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
    model5.layers[2].trainable = True
    optimizer = Adam(
        learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
        epsilon=1e-08,
        decay=0.01,
        clipnorm=1.0)
    loss =CategoricalCrossentropy(from_logits = True)
    metric = CategoricalAccuracy('balanced_accuracy'),
    model5.compile(
        optimizer = optimizer,
        loss = loss, 
        metrics = metric)
    tf.config.experimental_run_functions_eagerly(True)
    tf.config.run_functions_eagerly(True)

    train_history = model5.fit(
        x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
        y = to_categorical(data_train.Sentiment),
        validation_data = (
            {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.Sentiment)
            ),
       epochs=10,
       batch_size=36
       )
    predicted_raw = model5.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})
    predicted_raw = model5.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})
    y_predicted = np.argmax(predicted_raw, axis = 1)
    print(classification_report(data_val.Sentiment, y_predicted))
    print(confusion_matrix(data_val.Sentiment,y_predicted))
    cf_matrix=confusion_matrix(data_val.Sentiment,y_predicted)
    if(len(lis3)==0):
        break


#################### 1 ###########################
2187
(17213, 2) (2787, 2)


  0%|          | 0/2787 [00:00<?, ?it/s]

  0%|          | 0/2787 [00:00<?, ?it/s]

  0%|          | 0/2787 [00:00<?, ?it/s]

  0%|          | 0/2787 [00:00<?, ?it/s]

...

  0%|          | 0/16724 [00:00<?, ?it/s]

  0%|          | 0/16724 [00:00<?, ?it/s]

  0%|          | 0/16724 [00:00<?, ?it/s]

The number of words in string are : 115 565
Epoch 1/10
64/64 [==============================] - 50s 772ms/step - loss: 0.1730 - balanced_accuracy: 0.9573 - val_loss: 0.2381 - val_balanced_accuracy: 0.9573
Epoch 2/10
64/64 [==============================] - 49s 770ms/step - loss: 0.0468 - balanced_accuracy: 0.9900 - val_loss: 0.2417 - val_balanced_accuracy: 0.9593
Epoch 3/10
64/64 [==============================] - 50s 777ms/step - loss: 0.0153 - balanced_accuracy: 0.9969 - val_loss: 0.2684 - val_balanced_accuracy: 0.9553
Epoch 4/10
64/64 [==============================] - 50s 784ms/step - loss: 0.0143 - balanced_accuracy: 0.9969 - val_loss: 0.2500 - val_balanced_accuracy: 0.9604
Epoch 5/10
64/64 [==============================] - 50s 783ms/step - loss: 0.0110 - balanced_accuracy: 0.9965 - val_loss: 0.2736 - val_balanced_accuracy: 0.9563
Epoch 6/10
64/64 [==============================] - 50s 783ms/step - loss: 0.0104 - balanced_accuracy: 0.9974 - val_loss: 0.2762 - val_balanced_accurac

IndexError: single positional indexer is out-of-bounds